In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy  as np
import pandas as pd

import os

DETDIR='results/Main/'
models= ['google','alex']
compid = {'google': 'comp4_1460de5d-d642-42d1-9df0-fe9db3773178',
          'alex'  : 'comp4_2bf10b1f-a0f9-4006-a809-f3d507372676'}
pars = ['','eminus','proton','pizero','muminus','gamma','piminus']

In [ ]:
detections = {'google' : [f for f in os.listdir(DETDIR) if f.startswith(compid['google'])],
              'alex'   : [f for f in os.listdir(DETDIR) if f.startswith(compid['alex'])] }

In [ ]:
pre_frames = {}
for model in models:
    frames = []
    for det in detections[model]:
        frames.append(pd.read_csv(os.path.join(DETDIR,det),
                                  delimiter=' ',names=['file','prob','xmin','ymin','xmax','ymax','pred_class']) )
    pre_frames[model] = frames

In [ ]:
df_frames = {}
for model in models:
    pred_df = pd.concat(pre_frames[model],ignore_index=True)
    df_frames[model] = pred_df

In [ ]:
#Check a single result
alex_df = df_frames['alex']
alex_df[alex_df['file'] == 'muminus008457'].sort_values(by='prob',ascending=False).iloc[0:5]

In [ ]:
alex_df.file.unique().size

In [ ]:
import re
truth = re.compile("^([a-z]+)[0-9]+")

def get_gt(row): 
    return pars.index(truth.search(row.file).group(1))

In [ ]:
# Go row by row and mark the groundtruth

for model in models:
    the_df = df_frames[model]
    the_df['gt'] = the_df.apply(get_gt,axis=1)

In [ ]:
ious = []
probs = []
im_df = {}
for model in models:
    pred_df = df_frames[model]
    im = {}
    for ix,image in enumerate(pred_df.file.unique()):
        df = pred_df[pred_df['file'] == image].sort_values(by='prob',ascending=False)
        top = df.iloc[0] # take only the top prediction
        
        im[image] = {'gt':top['gt'],'top_pred_class':top['pred_class'],'top_prob':top['prob']}

    im_df[model] = pd.DataFrame(im).T

In [ ]:
ar = lambda x, y : np.arange(0,x,y)

In [ ]:
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['font.family']='serif'

In [ ]:
inc = 0.01



npars = len(pars) - 1
for pred in xrange(npars):
    pred += 1
    a = {}
    c = 0
    fig,ax = plt.subplots(figsize=(15,6))
    for model in models:
        c+=1
        model_df = im_df[model]
        ax = plt.subplot(1,2,c)

        for i in xrange(npars):
            i += 1    
            res = []
            for j in ar(1,inc):
                bot = model_df[model_df['top_pred_class'] == pred].query('top_prob > {}'.format(j))
                top = bot.query('gt == {}'.format(i))
                bot = float(bot.values.size)
                top = float(top.values.size)
                r = top/bot if bot > 0.0 else 0.0
                res.append(r * 100.0)
                a[i] = res


        ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
                weights=[a[r] for r in a],
                stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
        ax.set_ylim(0,100)
        ax.legend(loc='best',fontsize=12)
        
        ax.set_title("Detection: {} - Model: {}".format(pars[pred],model),fontweight='bold')
        ax.set_ylabel("Percent Truth",fontweight='bold')
        ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_detection_truth_{}_{}.pdf'.format(model,pars[pred],npars), format='pdf', dpi=1000)
    plt.show()

In [ ]:
inc = 0.01

npars = len(pars) - 1
for pred in xrange(npars):
    pred += 1
    a = {}
    c = 0
    fig,ax = plt.subplots(figsize=(15,6))
    for model in models:
        c+=1
        model_df = im_df[model]
        ax = plt.subplot(1,2,c)

        for i in xrange(npars):
            i += 1    
            res = []
            for j in ar(1,inc):
                bot = model_df[model_df['gt'] == pred].query('top_prob > {}'.format(j))
                top = bot.query('top_pred_class == {}'.format(i))
                bot = float(bot.values.size)
                top = float(top.values.size)
                r = top/bot if bot > 0.0 else 0.0
                res.append(r * 100.0)
                a[i] = res

        ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
                weights=[a[r] for r in a],
                stacked=True,histtype='stepfilled',alpha=0.65,label=[pars[r] for r in a])
        ax.set_ylim(0,100)
        ax.legend(loc='best',fontsize=12)
        
        ax.set_title("Truth: {} - Model: {}".format(pars[pred],model),fontweight='bold')
        ax.set_ylabel("Percent Detected",fontweight='bold')
        ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_truth_detection_{}_{}.pdf'.format(model,pars[pred],npars), format='pdf', dpi=1000)
    plt.show()